In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
import torch, torchvision 
from PIL import Image
from global_vars import labels

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
df_infos2 = pd.read_csv('../saved/infos2.csv')

In [8]:
from torchvision import datasets, models, transforms
from myeval import binary_acc
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import GroupKFold
import time
from myeval import binary_acc_core

kf = GroupKFold(5)
train_indices, test_indices = next(kf.split(df_infos2, df_infos2[labels], df_infos2['ptID']))

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

def load_cwt_nn_model(model_saved_path='../saved/modelCWTFullBalancedWeightedLoss/modelCWTFull0_model.dict'):
    model = models.resnet50(pretrained=True)
#     for param in model.parameters():
#         param.requires_grad = False
    
    num_ftrs = model.fc.in_features
    # change the last output to 9 classes
    model.fc = nn.Linear(num_ftrs, 9)
    
    # load saved model
    model.load_state_dict(torch.load(model_saved_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [10]:
from myeval import agg_y_preds, binary_acc


# out type

In [6]:
data_imgs = []
for i in tqdm(range(len(df_infos2))):
    with Image.open('../saved/channel_imgs/0/'+str(i)+'.png') as img:
        data_imgs.append(img.copy())


KeyboardInterrupt: 

In [37]:
import pickle as pkl

with open('../saved/channel_imgs0.pkg', 'wb') as f:
    pickle.dump(data_imgs, f)

In [38]:
import time
st = time.time()
with open('../saved/channel_imgs0.pkg', 'rb') as f:
    data_imgs = pickle.load(f)
    
print((time.time()-st)/60, 'min')

0.24144199291865032 min


In [ ]:
data_imgs

In [9]:
from torch.utils.data import Dataset
import os
class MyImageDatasetEvaluate(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_imgs):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_imgs = data_imgs
        self.transform =  transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                         ])
    def __len__(self):
        return len(self.data_imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = self.data_imgs[idx]
        sample = self.transform(image)
        return sample
    
image_datasets = MyImageDatasetEvaluate(data_imgs)

In [12]:
def run_12ECG_classifier(data_imgs,model):
    # Use your classifier here to obtain a label and score for each class. 
    
    fDatas = []
    #infos =[]
    fData = filter_data(data[:,1000:], highcut=50.0)
    #fData = datas[idx][:,1000:]
    intervals = sep_rr_interval(fData[:3], height=0.2, distance=200, plot=False)

    # basic info
    #info = get_basic_info(header_datas[idx], labels)
    
    # get data
    #ptID = info[0]
    #print(str(idx) + ' ' + ptID)
    for i in range(len(intervals)):
        l, r = intervals[i]
        fDatas.append(fData[:,l:r])
        #infos.append(info)
            
    fDataRes=np.array([signal.resample(fData, 250, axis=1) for fData in fDatas])
    fDataReNorms=np.array([normalize(fDataRe) for fDataRe in fDataRes])
    
    X = torch.FloatTensor(fDataReNorms)
    
    try:
        output = model(X)
        current_score = np.mean(torch.sigmoid(output).data.numpy(), axis=0)
        current_label = np.rint(current_score).astype(int)
   # print(current_score.shape, current_label.shape)
    except:
        print(fDataRes, header_data)
        current_label = np.zeros((9,), dtype=int)
        current_score = np.zeros((9,))
    return current_label, current_score

In [14]:
model = load_cwt_nn_model()

In [18]:
y_trues = []
y_prob_maxs = []
y_prob_means = []
with torch.no_grad():
    model.eval()
    for _, test_patient_indices in tqdm(df_infos2.iloc[test_indices].groupby('ptID').indices.items()):
        y_true = df_infos2.iloc[test_patient_indices[0]][labels]
        y_trues.append(y_true)
        X_imgs_patient = torch.utils.data.Subset(image_datasets, test_patient_indices)
        loader = torch.utils.data.DataLoader(X_imgs_patient, batch_size=len(test_indices), shuffle = False)#sampler = sampler)

        output_patient = None
        for X_batch in loader:   
            output = model(X_batch)
            y_prob_max, y_prob_mean = agg_y_preds(output)
            y_prob_maxs.append(y_prob_max)
            y_prob_means.append(y_prob_mean)
            
            
    

In [19]:
auroc,auprc,acc,fmeasure,fbeta,gbeta = binary_acc(torch.stack(y_prob_maxs), torch.Tensor(y_trues))

In [20]:
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc,auprc,acc,fmeasure,fbeta,gbeta)
print(output_string) 

AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure
0.120|0.381|0.209|0.120|0.914|0.547


In [21]:
auroc,auprc,acc,fmeasure,fbeta,gbeta = binary_acc(torch.stack(y_prob_means), torch.Tensor(y_trues))

In [22]:
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc,auprc,acc,fmeasure,fbeta,gbeta)
print(output_string) 

AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure
0.120|0.381|0.209|0.120|0.919|0.665


# use drive format

In [11]:
df_infos3 = pd.read_csv('../saved/infos3.csv')

In [12]:
import pickle
with open('../saved/datas.pkl', 'rb') as f:
    datas = pickle.load(f)
with open('../saved/header_datas.pkl', 'rb') as f:
    header_datas = pickle.load(f)

In [13]:
import pywt
def cwt(signals,name='', chn=6, wavelet_type = 'morl', vis=False):
    #print(signals.shape)
    # signals: channels x time
    num_steps = signals.shape[1]
    x = np.arange(num_steps) * 0.002
    delta_t = x[1] - x[0] # 500Hz 0.02s
    #print(delta_t)
    #Freq (5, 100)
    if wavelet_type == 'morl':
        scales = np.linspace(8,200,100)
    elif wavelet_type == 'mexh':
        scales = np.linspace(5,50,100)
    elif wavelet_type == 'gaus8':
        scales = np.linspace(12,120,100)
    else: # cmor
        scales = np.linspace(12,120,100)
    

    coef, freqs = pywt.cwt(signals[chn], scales, wavelet_type, delta_t)
    coef2 = (coef-np.min(coef))/(np.max(coef) - np.min(coef))
    #np.save('../saved/cwt/'+name, coef)
    if vis:
        fig = plt.figure()
        ax = fig.add_subplot(211)
        plt.plot(x,signals[chn])

        ax = fig.add_subplot(212)
        #plt.matshow(coefs.astype(float)) 
        plt.imshow(coef.astype(float), aspect='auto', extent=(1, signals.shape[1], freqs[-1], freqs[0]), cmap='jet')
       # plt.colorbar()
        #print((freqs[0], freqs[-1]))
       # plt.yticks(freqs)
        plt.suptitle(name  + ' ' +str(freqs[-1]) + ' '+  str(freqs[0]))
        plt.ylabel('Frequency(Hz)')
        plt.xlabel('Time(ms)')
        plt.show()
        plt.close()
        
    return coef2

In [14]:
from global_vars import labels
def get_image_and_labels(data, header_data):
    cmap = plt.get_cmap('jet')
    resize = torchvision.transforms.Resize((244, 244))
    info = get_basic_info(header_data, labels)
    
    n_length = data.shape[1]
    n_segments = int((n_length)/1000)
    
    coef = cwt(data, str(i)+' '+str(info[3]), chn=0, vis=False)
    data_imgs = []
    for j in range(n_segments):
        s = j*1000
        rgba_img = cmap(coef[:,s:s+1000])
        data_img = Image.fromarray((rgba_img[:, :, :3] * 255).astype(np.uint8))
        data_imgs.append(resize(data_img))
        
    return data_imgs, info[4:]

In [15]:
from global_vars import labels
def get_basic_info(header_data, labels):
    tmp_hea = header_data[0].split(' ')
    ptID = tmp_hea[0]
    age = None
    sex = None
    label = None
    Dx = None
    for iline in header_data:
        if iline.startswith('#Age'):
            tmp_age = iline.split(': ')[1].strip()
            age = int(tmp_age if tmp_age != 'NaN' else 57)
        elif iline.startswith('#Sex'):
            tmp_sex = iline.split(': ')[1]
            if tmp_sex.strip()=='Female':
                sex =1
            else:
                sex=0
        elif iline.startswith('#Dx'):
            Dx = np.char.rstrip(iline.split(': ')[1].split(','))
            label = [ll in Dx for ll in labels]
            
    
    return [ptID, sex, age, ' '.join(Dx)] + label

In [ ]:
model = load_cwt_nn_model()
transform =  transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                         ])

y_prob_maxs = []
y_prob_means = []
labels = []
with torch.no_grad():
    for i in tqdm(np.unique(df_infos3.iloc[test_indices]['idx']), leave=False):#fDataReNorms.shape[0])): #
        data_imgs, label = get_image_and_labels(datas[i], header_datas[i])
        imgs_tensor = torch.stack([transform(data_img) for data_img in data_imgs])
        test_output = model(imgs_tensor)
        y_prob_max, y_prob_mean = agg_y_preds(test_output)
        y_prob_maxs.append(y_prob_max)
        y_prob_means.append(y_prob_mean)
        labels.append(label)

/home/yfeng/anaconda3/envs/physioNet/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [48]:
np.unique(df_infos3.iloc[test_indices]['idx'])

array([   2,   12,   13, ..., 6863, 6864, 6872])

In [ ]:

#model = load_cwt_nn_model()
transform =  transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                         ])

y_prob_maxs = []
y_prob_means = []
labels = []
for i in tqdm(test_indices, leave=False):#fDataReNorms.shape[0])): #
    data_imgs, label = get_image_and_labels(datas[i], header_datas[i])
    imgs_tensor = torch.stack([transform(data_img) for data_img in data_imgs])
    test_output = model(imgs_tensor)
    y_prob_max, y_prob_mean = agg_y_preds(test_output)
    y_prob_maxs.append(y_prob_max)
    y_prob_means.append(y_prob_mean)
    labels = label